# Advanced Image Processing: Multiple Filters Implementation
This notebook implements various smoothing and sharpening filters for a specific image.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import os
from PIL import Image

def load_image(path):
    try:
        img = cv2.imread(path)
        if img is None:
            raise ValueError(f'Unable to load image from {path}')
        return img
    except Exception as e:
        raise Exception(f'Error loading image: {str(e)}')

def show_images(original, processed, title1='Original', title2='Processed'):
    try:
        if original is None or processed is None:
            raise ValueError('Input images cannot be None')
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
        if len(original.shape) == 2:  # Grayscale image
            ax1.imshow(original, cmap='gray')
            ax2.imshow(processed, cmap='gray')
        else:  # Color image
            ax1.imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
            ax2.imshow(cv2.cvtColor(processed, cv2.COLOR_BGR2RGB))
        ax1.set_title(title1)
        ax2.set_title(title2)
        plt.show()
    except Exception as e:
        raise Exception(f'Error displaying images: {str(e)}')

In [ ]:
def validate_kernel_size(kernel_size):
    if not isinstance(kernel_size, int) or kernel_size < 1 or kernel_size % 2 == 0:
        raise ValueError('Kernel size must be an odd positive integer')

# Enhanced Smoothing Filters
class SmoothingFilters:
    @staticmethod
    def mean_filter(image, kernel_size=3):
        return cv2.blur(image, (kernel_size, kernel_size))

    @staticmethod
    def median_filter(image, kernel_size=3):
        return cv2.medianBlur(image, kernel_size)

    @staticmethod
    def gaussian_filter(image, kernel_size=3, sigma=0):
        return cv2.GaussianBlur(image, (kernel_size, kernel_size), sigma)

    @staticmethod
    def bilateral_filter(image, d=9, sigma_color=75, sigma_space=75):
        return cv2.bilateralFilter(image, d, sigma_color, sigma_space)

    @staticmethod
    def box_filter(image, kernel_size=3):
        return cv2.boxFilter(image, -1, (kernel_size, kernel_size))

    @staticmethod
    def stack_filter(image, kernel_size=3):
        return cv2.stackBlur(image, (kernel_size, kernel_size))

    @staticmethod
    def motion_blur(image, kernel_size=15):
        kernel = np.zeros((kernel_size, kernel_size))
        kernel[int((kernel_size-1)/2), :] = np.ones(kernel_size)
        kernel = kernel / kernel_size
        return cv2.filter2D(image, -1, kernel)

    @staticmethod
    def anisotropic_diffusion(image, iterations=10):
        img_float = image.astype('float32')
        for _ in range(iterations):
            laplacian = cv2.Laplacian(img_float, cv2.CV_32F)
            img_float += 0.1 * laplacian
        return np.clip(img_float, 0, 255).astype('uint8')

    @staticmethod
    def nlm_filter(image, h=10, template_window=7, search_window=21):
        return cv2.fastNlMeansDenoisingColored(image, None, h, h, template_window, search_window)

    @staticmethod
    def wavelet_denoising(image):
        import pywt
        coeffs = pywt.dwt2(image, 'haar')
        cA, (cH, cV, cD) = coeffs
        coeffs = cA, (cH * 0.5, cV * 0.5, cD * 0.5)
        return pywt.idwt2(coeffs, 'haar')

In [ ]:
# Enhanced Sharpening Filters
class SharpeningFilters:
    @staticmethod
    def laplacian_filter(image):
        return cv2.Laplacian(image, cv2.CV_64F).astype(np.uint8)

    @staticmethod
    def high_boost_filter(image, alpha=1.5):
        gaussian = cv2.GaussianBlur(image, (3, 3), 0)
        return cv2.addWeighted(image, alpha, gaussian, -0.5, 0)

    @staticmethod
    def sobel_filter(image):
        sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
        sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
        return cv2.magnitude(sobelx, sobely).astype(np.uint8)

    @staticmethod
    def prewitt_filter(image):
        kernelx = np.array([[1,1,1],[0,0,0],[-1,-1,-1]])
        kernely = np.array([[-1,0,1],[-1,0,1],[-1,0,1]])
        img_prewittx = cv2.filter2D(image, -1, kernelx)
        img_prewitty = cv2.filter2D(image, -1, kernely)
        return cv2.addWeighted(img_prewittx, 0.5, img_prewitty, 0.5, 0)

    @staticmethod
    def unsharp_masking(image, sigma=1.0, strength=1.5):
        gaussian = cv2.GaussianBlur(image, (0, 0), sigma)
        return cv2.addWeighted(image, 1.0 + strength, gaussian, -strength, 0)

    @staticmethod
    def roberts_cross(image):
        kernel_x = np.array([[1, 0], [0, -1]])
        kernel_y = np.array([[0, 1], [-1, 0]])
        return np.abs(cv2.filter2D(image, -1, kernel_x)) + np.abs(cv2.filter2D(image, -1, kernel_y))

    @staticmethod
    def scharr_filter(image):
        scharrx = cv2.Scharr(image, cv2.CV_64F, 1, 0)
        scharry = cv2.Scharr(image, cv2.CV_64F, 0, 1)
        return cv2.magnitude(scharrx, scharry).astype(np.uint8)

    @staticmethod
    def dog_filter(image, sigma1=1, sigma2=2):
        g1 = cv2.GaussianBlur(image, (0, 0), sigma1)
        g2 = cv2.GaussianBlur(image, (0, 0), sigma2)
        return cv2.subtract(g1, g2)

    @staticmethod
    def emboss_filter(image):
        kernel = np.array([[-2,-1,0], [-1,1,1], [0,1,2]])
        return cv2.filter2D(image, -1, kernel) + 128

    @staticmethod
    def kirsch_compass_filter(image):
        kirsch = np.zeros_like(image)
        for angle in range(8):
            k = np.zeros((3, 3), dtype=np.float32)
            k.fill(-3)
            k[1, 1] = 0
            for i in range(3):
                k[i, 2] = 5
            k = np.rot90(k, angle)
            kirsch = np.maximum(kirsch, cv2.filter2D(image, -1, k))
        return kirsch.astype(np.uint8)

In [ ]:
def process_single_image(image_path):
    try:
        # Initialize filter classes
        smoothing = SmoothingFilters()
        sharpening = SharpeningFilters()

        # Load the image
        img = load_image(image_path)
        if img is None:
            raise ValueError('Failed to load image')

        print(f'Processing {os.path.basename(image_path)}')

        # Define filter methods with their names
        smooth_methods = [
            (smoothing.mean_filter, 'Mean'),
            (smoothing.median_filter, 'Median'),
            (smoothing.gaussian_filter, 'Gaussian'),
            (smoothing.bilateral_filter, 'Bilateral'),
            (smoothing.box_filter, 'Box'),
            (smoothing.motion_blur, 'Motion Blur'),
            (smoothing.anisotropic_diffusion, 'Anisotropic'),
            (smoothing.nlm_filter, 'Non-Local Means')
        ]

        sharp_methods = [
            (sharpening.laplacian_filter, 'Laplacian'),
            (sharpening.high_boost_filter, 'High Boost'),
            (sharpening.sobel_filter, 'Sobel'),
            (sharpening.prewitt_filter, 'Prewitt'),
            (sharpening.unsharp_masking, 'Unsharp Mask'),
            (sharpening.roberts_cross, 'Roberts Cross'),
            (sharpening.scharr_filter, 'Scharr'),
            (sharpening.dog_filter, 'DoG'),
            (sharpening.emboss_filter, 'Emboss'),
            (sharpening.kirsch_compass_filter, 'Kirsch Compass')
        ]

        # Process and display each filter result
        for method, name in smooth_methods + sharp_methods:
            try:
                processed = method(img)
                show_images(img, processed, 'Original', f'{name} Filter')
            except Exception as e:
                print(f'Error applying {name} filter: {str(e)}')

    except Exception as e:
        print(f'Error processing image: {str(e)}')

# Process the specific image
image_path = r'F:\BFCAI\image processing\Project\Images\brightness.jpg'
process_single_image(image_path)